In [ ]:
# Toutes les colonnes nécessaires pour les calculs actuels

NECESSARY_COLS = {
    'ford': ['vin', 'soc', 'battery_energy', 'odometer', 'status', 'trimmed_in_discharge_idx', 'net_capacity', 'range'],
    'renault':['vin', 'soc', 'battery_energy', 'net_capacity', 'odometer', 'range', 'in_charge', 'charging_rate'],
    'kia':[],
    'volvo-cars': ['vin', 'date', 'odometer', 'range', 'soc', 'estimated_range'],
    'tesla-fleet-telemetry': ['vin', 'in_charge_idx', 'ac_charge_energy_added', 'dc_charge_energy_added', 'soc', 'net_capacity', 'odometer', 'range', 'ac_charging_power', 'dc_charging_power'],
    'volkswagen':[],
    'bmw': [],
    'mercedes-benz': ['vin', 'date', 'soc', 'odometer', 'range', 'model', 'capacity', 'in_charge', 'in_discharge', 'trimmed_in_charge', 'trimmed_in_discharge_idx'],
    'stellantis': ['vin', 'date', 'odometer', 'range', 'soh_oem'],
}

In [ ]:
from core.spark_utils import create_spark_session
from core.s3.settings import S3Settings
from core.s3.s3_utils import S3Service

settings = S3Settings()

spark = create_spark_session(
    settings.S3_KEY,
    settings.S3_SECRET
)

s3_service = S3Service()

for oem, cols in NECESSARY_COLS.items():
    columns = s3_service.read_parquet_df_spark(spark, f'raw_ts/{oem}/time_series/raw_ts_spark.parquet').columns
    print(f"{oem}: {True if 'consumption' in columns else False}")

In [ ]:
from core.spark_utils import create_spark_session
from core.s3.settings import S3Settings
from core.s3.s3_utils import S3Service

settings = S3Settings()

spark = create_spark_session(
    settings.S3_KEY,
    settings.S3_SECRET
)

s3_service = S3Service()

for oem, cols in NECESSARY_COLS.items():
    columns = s3_service.read_parquet_df_spark(spark, f'processed_ts/{oem}/time_series/processed_ts_spark.parquet').columns
    missing_cols = set(cols) - set(columns)
    additional_cols = set(columns) - set(cols)
    print(f"{oem}: {True if 'consumption' in columns else False}")
    print(f'{oem}: missing_cols: {missing_cols}')
    print(f'{oem}: additional_cols: {additional_cols}')

In [ ]:
columns = s3_service.read_parquet_df_spark(spark, f'processed_ts/ford/time_series/processed_ts_spark.parquet')

columns.show()

In [ ]:
# Qu'est ce qui est mutualisable dans une classe ?


# FORD 

# SOH Ford Process -
# On détache le 9ème décile de la battery_energy
# On l'ajoute à chaque ligne le tss 
# Le soh devient battery_energy / max_battery_energy

# Consumption Ford Process 
# On groupe par période de décharge et vin 
# On prend premier et dernier soc, premier et dernier odometer et la net capacitu
# consumption=soc_diff * net_capacity * 100 / odometer_diff 
# On joint sur l'index de décharge

# Il n'y a donc pas encore d'aggrégation à cette étape

# RENAULT

# SOH Renault process
#  On calcule l'expected_battery_energy = net_capacity * soc
#  Pour calculer le soh = battery_energy / expected_battery_energy

# Charge levels Renault process
# On calcule les types de charge via le charging_rate et des seuils determinés par nos soins en comaparant à la soc_diff
# .eval("level_1 = soc_diff * (charging_rate < @LEVEL_1_MAX_POWER)")
# .eval("level_2 = soc_diff * (charging_rate.between(@LEVEL_1_MAX_POWER, @LEVEL_2_MAX_POWER))")
# .eval("level_3 = soc_diff * (charging_rate > @LEVEL_2_MAX_POWER)")
# # Il n'y a donc pas encore d'aggrégation à cette étape non plus

# TESLA FT
# Groupement par VIN et indice de charge
# ac_energy_added_min = min ac_charge_energy_added
# dc_energy_added_min = min dc_charge_energy_added
# ac_energy_added_end= last ac_charge_energy_added
# dc_energy_added_end= last dc_charge_energy_added
# soc_diff= différence sur le group by
# inside_temp= valeur moyenne
# net_capacity= first (mais peu importe)
# range= first (mais peu importe)
# odometer= first (mais peu importe car période de charge)
# version= first (mais peu importe)
# size= compte le nombre d'enregistrements de soc
# model = first (mais peu importe)
# date= first 
# ac_charging_power= médiane
# dc_charging_power= médiane
# tesla_code = first (mais peu importe)

# charging power = ac_charging_power + dc_charging_power
# ac_energy_added = ac_energy_added_end - ac_energy_added_min
# dc_energy_added = dc_energy_added_end - dc_energy_added_min
# energy_added = dc_energy_added
# soh = energy_added / (soc_diff / 100.0 * net_capacity)
# level_1 = soc_diff * (charging_power < 8) / 100
# level_2 = soc_diff * (charging_power.between(8, 45)) / 100
# level_3 = soc_diff * (charging_power > 45) / 100

# Il y a aggrégation par VIN et indice de charge à cette étape

# MERCEDES
# OUTDATED

# STELLANTIS

# Petit forward ill et backward fill de l'odometer
# On groupe par vin et jour
# Odometer : last
# SOH OEM : median

# Aggrégation par vin et jour

# VOLVO
# Petit forward ill et backward fill de l'odometer
# On groupe par vin et jour
# soh = estimated_range / soc / range / 0.8

# Aggrégation par vin et jour

In [ ]:
from core.spark_utils import create_spark_session
from core.s3.settings import S3Settings
from core.s3.s3_utils import S3Service

settings = S3Settings()

spark = create_spark_session(
    settings.S3_KEY,
    settings.S3_SECRET
)

s3_service = S3Service()

df = s3_service.read_parquet_df_spark(spark, 'raw_results/raw_results_stellantis_spark.parquet')

volvo = df.toPandas()

In [ ]:
volvo